In [54]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install --upgrade sklearn

Requirement already up-to-date: sklearn in c:\users\anastasia\anaconda3\lib\site-packages (0.0)
Note: you may need to restart the kernel to use updated packages.


In [56]:
# подключаем библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import numpy.ma as ma
from sklearn import base
from sklearn.model_selection import KFold
# from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from scipy import stats
import json

import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [58]:
print ("Train data shape:", train.shape)
print ("Test data shape:", test.shape)

Train data shape: (1460, 81)
Test data shape: (1459, 80)


In [59]:
train.head(20
        )

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [60]:
# impute the missing values and create the missing value indicator variables for each numeric column.
train_numeric = train.select_dtypes(include=[np.number])
numeric_cols = train_numeric.columns.values

for col in numeric_cols:
    missing = train[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        train['{}_ismissing'.format(col)] = missing
        med = train[col].median()
        train[col] = train[col].fillna(med)

imputing missing values for: LotFrontage
imputing missing values for: MasVnrArea
imputing missing values for: GarageYrBlt


In [61]:
train.dtypes

Id                         int64
MSSubClass                 int64
MSZoning                  object
LotFrontage              float64
LotArea                    int64
                          ...   
SaleCondition             object
SalePrice                  int64
LotFrontage_ismissing       bool
MasVnrArea_ismissing        bool
GarageYrBlt_ismissing       bool
Length: 84, dtype: object

In [62]:

cols_to_drop = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_less = train.drop(cols_to_drop, axis=1, inplace=True)

In [63]:
# Процентный список пропущенных данных
for col in train.columns:
    pct_missing = np.mean(train[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))
    
    

Id - 0.0%
MSSubClass - 0.0%
MSZoning - 0.0%
LotFrontage - 0.0%
LotArea - 0.0%
Street - 0.0%
LotShape - 0.0%
LandContour - 0.0%
Utilities - 0.0%
LotConfig - 0.0%
LandSlope - 0.0%
Neighborhood - 0.0%
Condition1 - 0.0%
Condition2 - 0.0%
BldgType - 0.0%
HouseStyle - 0.0%
OverallQual - 0.0%
OverallCond - 0.0%
YearBuilt - 0.0%
YearRemodAdd - 0.0%
RoofStyle - 0.0%
RoofMatl - 0.0%
Exterior1st - 0.0%
Exterior2nd - 0.0%
MasVnrType - 1.0%
MasVnrArea - 0.0%
ExterQual - 0.0%
ExterCond - 0.0%
Foundation - 0.0%
BsmtQual - 3.0%
BsmtCond - 3.0%
BsmtExposure - 3.0%
BsmtFinType1 - 3.0%
BsmtFinSF1 - 0.0%
BsmtFinType2 - 3.0%
BsmtFinSF2 - 0.0%
BsmtUnfSF - 0.0%
TotalBsmtSF - 0.0%
Heating - 0.0%
HeatingQC - 0.0%
CentralAir - 0.0%
Electrical - 0.0%
1stFlrSF - 0.0%
2ndFlrSF - 0.0%
LowQualFinSF - 0.0%
GrLivArea - 0.0%
BsmtFullBath - 0.0%
BsmtHalfBath - 0.0%
FullBath - 0.0%
HalfBath - 0.0%
BedroomAbvGr - 0.0%
KitchenAbvGr - 0.0%
KitchenQual - 0.0%
TotRmsAbvGrd - 0.0%
Functional - 0.0%
Fireplaces - 0.0%
GarageType

In [64]:

#One-hot encoding

In [65]:
train= train.dropna()

In [66]:
categorical_columns = train.columns[train.dtypes == 'object']


In [67]:

onehot_encoder = OneHotEncoder(sparse=False)

encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(train[categorical_columns]))
encoded_categorical_columns.head()

,0,1,2,3,4,5,6,7,8,9,...,219,220,221,222,223,224,225,226,227,228
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [68]:
# train_test_split

In [69]:
train.dtypes

Id                         int64
MSSubClass                 int64
MSZoning                  object
LotFrontage              float64
LotArea                    int64
                          ...   
SaleCondition             object
SalePrice                  int64
LotFrontage_ismissing       bool
MasVnrArea_ismissing        bool
GarageYrBlt_ismissing       bool
Length: 79, dtype: object

In [70]:
X = train.drop(['SalePrice', 'Id'],axis=1)
y = train['SalePrice']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [72]:
# Распределение фичей

In [73]:
test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [74]:
# catboost

In [75]:
from catboost import CatBoostRegressor

In [76]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [77]:

model= CatBoostRegressor(iterations=2000,learning_rate=1,depth = 2)

In [78]:
model.fit(X_train,y_train,categorical_features_indices)

0:	learn: 60798.5433771	total: 15.7ms	remaining: 31.5s
1:	learn: 53290.9985281	total: 31.5ms	remaining: 31.4s
2:	learn: 50900.9615724	total: 47ms	remaining: 31.3s
3:	learn: 46486.9313714	total: 58.8ms	remaining: 29.3s
4:	learn: 44507.6279445	total: 71.8ms	remaining: 28.6s
5:	learn: 42519.7272484	total: 83ms	remaining: 27.6s
6:	learn: 41272.3362202	total: 93.7ms	remaining: 26.7s
7:	learn: 39120.0415661	total: 103ms	remaining: 25.7s
8:	learn: 37598.4537506	total: 117ms	remaining: 25.8s
9:	learn: 37040.1146749	total: 127ms	remaining: 25.3s
10:	learn: 36269.9227888	total: 141ms	remaining: 25.5s
11:	learn: 35301.3783397	total: 152ms	remaining: 25.2s
12:	learn: 35089.6670979	total: 165ms	remaining: 25.3s
13:	learn: 35070.2088746	total: 181ms	remaining: 25.7s
14:	learn: 34757.6958113	total: 195ms	remaining: 25.8s
15:	learn: 34036.3397676	total: 207ms	remaining: 25.7s
16:	learn: 33297.0625637	total: 221ms	remaining: 25.8s
17:	learn: 32976.4524127	total: 232ms	remaining: 25.6s
18:	learn: 32213.

164:	learn: 20478.4890284	total: 1.81s	remaining: 20.1s
165:	learn: 20401.7925484	total: 1.82s	remaining: 20.1s
166:	learn: 20397.8191134	total: 1.83s	remaining: 20.1s
167:	learn: 20380.9266410	total: 1.84s	remaining: 20s
168:	learn: 20354.5222335	total: 1.85s	remaining: 20s
169:	learn: 20349.2497505	total: 1.85s	remaining: 20s
170:	learn: 20325.5922696	total: 1.86s	remaining: 19.9s
171:	learn: 20280.6057048	total: 1.87s	remaining: 19.9s
172:	learn: 20263.0042454	total: 1.88s	remaining: 19.9s
173:	learn: 20203.1551725	total: 1.89s	remaining: 19.8s
174:	learn: 20190.1091998	total: 1.9s	remaining: 19.8s
175:	learn: 20131.2256826	total: 1.91s	remaining: 19.8s
176:	learn: 20082.3613853	total: 1.92s	remaining: 19.7s
177:	learn: 20054.3567415	total: 1.93s	remaining: 19.7s
178:	learn: 20042.1303644	total: 1.93s	remaining: 19.7s
179:	learn: 20008.7382440	total: 1.94s	remaining: 19.6s
180:	learn: 19979.6171590	total: 1.95s	remaining: 19.6s
181:	learn: 19904.9578062	total: 1.96s	remaining: 19.6s

320:	learn: 16889.9482136	total: 3.24s	remaining: 17s
321:	learn: 16854.1856077	total: 3.25s	remaining: 17s
322:	learn: 16832.8033298	total: 3.26s	remaining: 16.9s
323:	learn: 16771.7671163	total: 3.27s	remaining: 16.9s
324:	learn: 16767.9480264	total: 3.28s	remaining: 16.9s
325:	learn: 16764.7799843	total: 3.29s	remaining: 16.9s
326:	learn: 16752.4639470	total: 3.3s	remaining: 16.9s
327:	learn: 16739.8262610	total: 3.31s	remaining: 16.9s
328:	learn: 16731.2204709	total: 3.32s	remaining: 16.9s
329:	learn: 16728.1557237	total: 3.33s	remaining: 16.9s
330:	learn: 16672.8009710	total: 3.34s	remaining: 16.8s
331:	learn: 16656.2299554	total: 3.35s	remaining: 16.8s
332:	learn: 16629.0396944	total: 3.36s	remaining: 16.8s
333:	learn: 16596.5844547	total: 3.37s	remaining: 16.8s
334:	learn: 16594.2155220	total: 3.38s	remaining: 16.8s
335:	learn: 16587.9095057	total: 3.39s	remaining: 16.8s
336:	learn: 16577.5586147	total: 3.4s	remaining: 16.8s
337:	learn: 16540.2666484	total: 3.41s	remaining: 16.8

474:	learn: 14320.5653818	total: 4.69s	remaining: 15s
475:	learn: 14303.3883470	total: 4.7s	remaining: 15s
476:	learn: 14295.5532932	total: 4.7s	remaining: 15s
477:	learn: 14293.5378680	total: 4.71s	remaining: 15s
478:	learn: 14267.6140761	total: 4.72s	remaining: 15s
479:	learn: 14251.0972429	total: 4.73s	remaining: 15s
480:	learn: 14221.8379337	total: 4.74s	remaining: 15s
481:	learn: 14211.8935092	total: 4.75s	remaining: 15s
482:	learn: 14196.2230994	total: 4.76s	remaining: 14.9s
483:	learn: 14163.4221728	total: 4.77s	remaining: 14.9s
484:	learn: 14156.9423564	total: 4.78s	remaining: 14.9s
485:	learn: 14147.0385495	total: 4.78s	remaining: 14.9s
486:	learn: 14145.2741100	total: 4.79s	remaining: 14.9s
487:	learn: 14133.1273149	total: 4.8s	remaining: 14.9s
488:	learn: 14125.4925050	total: 4.81s	remaining: 14.9s
489:	learn: 14119.8738380	total: 4.82s	remaining: 14.9s
490:	learn: 14096.6914251	total: 4.83s	remaining: 14.8s
491:	learn: 14084.7101180	total: 4.84s	remaining: 14.8s
492:	learn:

632:	learn: 12477.6701130	total: 6.12s	remaining: 13.2s
633:	learn: 12476.2502157	total: 6.13s	remaining: 13.2s
634:	learn: 12467.7069593	total: 6.14s	remaining: 13.2s
635:	learn: 12466.6246370	total: 6.15s	remaining: 13.2s
636:	learn: 12454.9387680	total: 6.16s	remaining: 13.2s
637:	learn: 12453.9851009	total: 6.16s	remaining: 13.2s
638:	learn: 12445.2057961	total: 6.17s	remaining: 13.2s
639:	learn: 12438.6660650	total: 6.18s	remaining: 13.1s
640:	learn: 12435.9403971	total: 6.19s	remaining: 13.1s
641:	learn: 12414.8182868	total: 6.2s	remaining: 13.1s
642:	learn: 12401.2312212	total: 6.21s	remaining: 13.1s
643:	learn: 12382.8893010	total: 6.22s	remaining: 13.1s
644:	learn: 12365.0713366	total: 6.23s	remaining: 13.1s
645:	learn: 12364.3793809	total: 6.24s	remaining: 13.1s
646:	learn: 12364.1019854	total: 6.25s	remaining: 13.1s
647:	learn: 12350.9383735	total: 6.25s	remaining: 13.1s
648:	learn: 12343.9579956	total: 6.26s	remaining: 13s
649:	learn: 12329.8389558	total: 6.27s	remaining: 1

...

1267:	learn: 7509.3101251	total: 12.6s	remaining: 7.27s
1268:	learn: 7508.5526381	total: 12.6s	remaining: 7.26s
1269:	learn: 7507.9304683	total: 12.6s	remaining: 7.25s
1270:	learn: 7506.3525428	total: 12.6s	remaining: 7.24s
1271:	learn: 7501.7081252	total: 12.6s	remaining: 7.23s
1272:	learn: 7499.0585226	total: 12.6s	remaining: 7.22s
1273:	learn: 7458.4039563	total: 12.7s	remaining: 7.21s
1274:	learn: 7458.4036859	total: 12.7s	remaining: 7.2s
1275:	learn: 7437.7825914	total: 12.7s	remaining: 7.19s
1276:	learn: 7435.4578710	total: 12.7s	remaining: 7.18s
1277:	learn: 7428.9225467	total: 12.7s	remaining: 7.17s
1278:	learn: 7426.8593655	total: 12.7s	remaining: 7.17s
1279:	learn: 7426.8481238	total: 12.7s	remaining: 7.16s
1280:	learn: 7426.7639063	total: 12.7s	remaining: 7.15s
1281:	learn: 7421.2862622	total: 12.7s	remaining: 7.14s
1282:	learn: 7412.3862296	total: 12.8s	remaining: 7.13s
1283:	learn: 7411.3801934	total: 12.8s	remaining: 7.12s
1284:	learn: 7405.1678144	total: 12.8s	remaining:

1429:	learn: 6806.9611797	total: 14.2s	remaining: 5.66s
1430:	learn: 6803.3685015	total: 14.2s	remaining: 5.65s
1431:	learn: 6793.6166129	total: 14.2s	remaining: 5.64s
1432:	learn: 6785.4993685	total: 14.2s	remaining: 5.63s
1433:	learn: 6781.1604572	total: 14.2s	remaining: 5.62s
1434:	learn: 6777.8157879	total: 14.2s	remaining: 5.61s
1435:	learn: 6777.5250524	total: 14.3s	remaining: 5.6s
1436:	learn: 6774.2514735	total: 14.3s	remaining: 5.59s
1437:	learn: 6772.4362872	total: 14.3s	remaining: 5.58s
1438:	learn: 6771.5600753	total: 14.3s	remaining: 5.57s
1439:	learn: 6768.1499685	total: 14.3s	remaining: 5.56s
1440:	learn: 6761.4271314	total: 14.3s	remaining: 5.55s
1441:	learn: 6760.5787444	total: 14.3s	remaining: 5.54s
1442:	learn: 6758.1396387	total: 14.3s	remaining: 5.53s
1443:	learn: 6752.8204096	total: 14.3s	remaining: 5.52s
1444:	learn: 6739.2405283	total: 14.3s	remaining: 5.51s
1445:	learn: 6734.7974799	total: 14.3s	remaining: 5.5s
1446:	learn: 6731.6365017	total: 14.4s	remaining: 

1583:	learn: 6137.8062202	total: 15.6s	remaining: 4.11s
1584:	learn: 6129.4088910	total: 15.7s	remaining: 4.1s
1585:	learn: 6125.7190128	total: 15.7s	remaining: 4.09s
1586:	learn: 6124.1575308	total: 15.7s	remaining: 4.08s
1587:	learn: 6119.7614156	total: 15.7s	remaining: 4.07s
1588:	learn: 6118.3188902	total: 15.7s	remaining: 4.06s
1589:	learn: 6108.8100644	total: 15.7s	remaining: 4.05s
1590:	learn: 6079.3602917	total: 15.7s	remaining: 4.04s
1591:	learn: 6077.4156606	total: 15.7s	remaining: 4.03s
1592:	learn: 6076.4590233	total: 15.7s	remaining: 4.02s
1593:	learn: 6072.2490014	total: 15.7s	remaining: 4.01s
1594:	learn: 6069.3714315	total: 15.8s	remaining: 4s
1595:	learn: 6067.7473417	total: 15.8s	remaining: 3.99s
1596:	learn: 6065.7821558	total: 15.8s	remaining: 3.98s
1597:	learn: 6064.8602784	total: 15.8s	remaining: 3.97s
1598:	learn: 6062.7318943	total: 15.8s	remaining: 3.96s
1599:	learn: 6061.6478043	total: 15.8s	remaining: 3.95s
1600:	learn: 6058.6086189	total: 15.8s	remaining: 3.

1747:	learn: 5483.6959495	total: 17.3s	remaining: 2.49s
1748:	learn: 5483.5979677	total: 17.3s	remaining: 2.48s
1749:	learn: 5482.7832502	total: 17.3s	remaining: 2.47s
1750:	learn: 5481.9219396	total: 17.3s	remaining: 2.46s
1751:	learn: 5476.4185284	total: 17.3s	remaining: 2.45s
1752:	learn: 5475.3128918	total: 17.3s	remaining: 2.44s
1753:	learn: 5472.5101149	total: 17.3s	remaining: 2.43s
1754:	learn: 5468.5101893	total: 17.3s	remaining: 2.42s
1755:	learn: 5459.4840158	total: 17.3s	remaining: 2.41s
1756:	learn: 5458.7664352	total: 17.3s	remaining: 2.4s
1757:	learn: 5451.6107270	total: 17.4s	remaining: 2.39s
1758:	learn: 5450.7244475	total: 17.4s	remaining: 2.38s
1759:	learn: 5446.2397954	total: 17.4s	remaining: 2.37s
1760:	learn: 5440.4084250	total: 17.4s	remaining: 2.36s
1761:	learn: 5440.0093015	total: 17.4s	remaining: 2.35s
1762:	learn: 5438.8919849	total: 17.4s	remaining: 2.34s
1763:	learn: 5436.2529359	total: 17.4s	remaining: 2.33s
1764:	learn: 5431.8512782	total: 17.4s	remaining:

1897:	learn: 5034.5288320	total: 18.7s	remaining: 1s
1898:	learn: 5033.8008555	total: 18.7s	remaining: 995ms
1899:	learn: 5031.5436258	total: 18.7s	remaining: 985ms
1900:	learn: 5027.5733140	total: 18.7s	remaining: 975ms
1901:	learn: 5025.8940658	total: 18.7s	remaining: 965ms
1902:	learn: 5024.5679997	total: 18.7s	remaining: 955ms
1903:	learn: 5022.8237786	total: 18.8s	remaining: 946ms
1904:	learn: 5018.4224162	total: 18.8s	remaining: 936ms
1905:	learn: 5016.4134581	total: 18.8s	remaining: 926ms
1906:	learn: 5015.7231076	total: 18.8s	remaining: 916ms
1907:	learn: 5014.0950885	total: 18.8s	remaining: 906ms
1908:	learn: 5014.0935339	total: 18.8s	remaining: 896ms
1909:	learn: 5013.0239724	total: 18.8s	remaining: 886ms
1910:	learn: 5011.5764159	total: 18.8s	remaining: 876ms
1911:	learn: 5011.3010557	total: 18.8s	remaining: 867ms
1912:	learn: 5009.3343468	total: 18.8s	remaining: 857ms
1913:	learn: 5007.4474819	total: 18.8s	remaining: 847ms
1914:	learn: 5007.4136022	total: 18.9s	remaining: 8

In [79]:
# MAE metrics

In [80]:
from sklearn.metrics import mean_absolute_error as mae

In [81]:
#MAE with OHE = 17771.99771484267
#MAE with TE = 

In [82]:
 predictions = model.predict(X_test)
 MAE = mae(predictions, y_test)

In [83]:
print(str(MAE))
    

27640.063893391678


In [84]:
class KFoldTargetEncoderTrain(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, colnames,targetName,n_fold=5,verbosity=True,discardOriginal_col=False):

        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col

    def fit(self, X, y=None):
        return self


    def transform(self,X):

        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)

        mean_of_target = X[self.targetName].mean()
        kf = KFold(n_splits = self.n_fold, shuffle = False, random_state=2019)



        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan

        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
#             print(tr_ind,val_ind)
            X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].map(X_tr.groupby(self.colnames)[self.targetName].mean())

        X[col_mean_name].fillna(mean_of_target, inplace = True)

        if self.verbosity:

            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,
                                                                                      self.targetName,
                                                                                      np.corrcoef(X[self.targetName].values, encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.colnames, axis=1)
            

        return X

In [85]:
categorical_columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [86]:

# cols_to_drop2 = ['HouseStyle_Kfold_Target_Enc','RoofStyle_Kfold_Target_Enc','RoofMatl_Kfold_Target_Enc','Exterior2nd_Kfold_Target_Enc']
# categorical_columns = X_te_test.drop(cols_to_drop2, axis=1, inplace=True) 

In [87]:
categorical_columns.append(pd.Index(["Exterior1st_Kfold_Target_Enc","BsmtQual_Kfold_Target_Enc","BsmtCond_Kfold_Target_Enc","BsmtExposure_Kfold_Target_Enc","BsmtFinType1_Kfold_Target_Enc","BsmtFinType2_Kfold_Target_Enc","Heating_Kfold_Target_Enc"]))

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition',
       'Exterior1st_Kfold_Target_Enc', 'BsmtQual_Kfold_Target_Enc',
       'BsmtCond_Kfold_Target_Enc', 'BsmtExposure_Kfold_Target_Enc',
       'BsmtFinType1_Kfold_Target_Enc', 'BsmtFinType2_Kfold_Target_Enc',
       'Heating_Kfold_Target_Enc'],
      dtype='object')

In [88]:
Xy_train = pd.concat([X_train,y_train], 1)
Xy_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_ismissing,MasVnrArea_ismissing,GarageYrBlt_ismissing,SalePrice
1145,50,RM,52.0,6240,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,8,2006,WD,Family,False,False,False,149000
745,60,RL,69.0,8963,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,7,2008,WD,Normal,True,False,False,299800
1134,60,RL,57.0,8773,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0,0,8,2007,WD,Normal,False,False,False,169000
537,20,RL,69.0,12735,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,4,2008,COD,Normal,True,False,False,111250
41,20,RL,115.0,16905,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,7,2007,WD,Normal,False,False,False,170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,60,RL,58.0,14054,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,11,2006,New,Partial,False,False,False,219210
1236,160,RL,36.0,2628,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,6,2010,WD,Normal,False,False,False,175500
1413,20,RL,88.0,10994,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,9,2009,COD,Abnorml,False,False,False,257000
933,20,RL,63.0,8487,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,8,2009,WD,Normal,False,False,False,190000


In [89]:
for col in categorical_columns:
    Xy_train = KFoldTargetEncoderTrain(colnames=col, targetName="SalePrice", discardOriginal_col=False).transform(Xy_train)

Correlation between the new feature, MSZoning_Kfold_Target_Enc and, SalePrice is 0.29897378946135816.
Correlation between the new feature, Street_Kfold_Target_Enc and, SalePrice is 0.04467542258125284.
Correlation between the new feature, LotShape_Kfold_Target_Enc and, SalePrice is 0.2608805754931189.
Correlation between the new feature, LandContour_Kfold_Target_Enc and, SalePrice is 0.09950208170866165.
Correlation between the new feature, Utilities_Kfold_Target_Enc and, SalePrice is -0.07838505956187564.
Correlation between the new feature, LotConfig_Kfold_Target_Enc and, SalePrice is 0.10000446121706559.
Correlation between the new feature, LandSlope_Kfold_Target_Enc and, SalePrice is 0.011401489409827548.
Correlation between the new feature, Neighborhood_Kfold_Target_Enc and, SalePrice is 0.7043828032970537.
Correlation between the new feature, Condition1_Kfold_Target_Enc and, SalePrice is 0.13110944231043647.
Correlation between the new feature, Condition2_Kfold_Target_Enc and, Sa

In [90]:
class KFoldTargetEncoderTest(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,encodedName):
        
        self.train = train
        self.colNames = colNames
        self.encodedName = encodedName
        
        
    def fit(self, X, y=None):
        return self

    def transform(self,X):


        mean = self.train[[self.colNames,self.encodedName]].groupby(self.colNames).mean().reset_index() 
        
        dd = {}
        for index, row in mean.iterrows():
            dd[row[self.colNames]] = row[self.encodedName]

        
        X[self.encodedName] = X[self.colNames]
        X = X.replace({self.encodedName: dd})

        return X
        

In [91]:
X_te_test = X_test.copy()

In [92]:
for col in categorical_columns:
    X_te_test = KFoldTargetEncoderTest(train=Xy_train, colNames=col, encodedName=col + '_' + 'Kfold_Target_Enc').transform(X_te_test)

In [93]:
for col in categorical_columns:
    Xy_train = Xy_train.drop(col, 1)

In [94]:
for col in categorical_columns:
    X_te_test = X_te_test.drop(col, 1)

In [95]:
model_te = CatBoostRegressor(iterations=2000,learning_rate=1,depth = 2)

In [96]:
model.fit(Xy_train.drop('SalePrice', axis=1),Xy_train['SalePrice'])

0:	learn: 58416.8905014	total: 2.61ms	remaining: 5.23s
1:	learn: 49104.3758592	total: 4.12ms	remaining: 4.12s
2:	learn: 45905.6007129	total: 5.31ms	remaining: 3.54s
3:	learn: 40503.6601548	total: 6.48ms	remaining: 3.23s
4:	learn: 37623.6228041	total: 7.68ms	remaining: 3.06s
5:	learn: 35865.7039728	total: 9.24ms	remaining: 3.07s
6:	learn: 33847.0366093	total: 10.4ms	remaining: 2.96s
7:	learn: 32483.9019184	total: 11.6ms	remaining: 2.88s
8:	learn: 30829.8296018	total: 12.7ms	remaining: 2.81s
9:	learn: 29415.6607275	total: 14.7ms	remaining: 2.93s
10:	learn: 27975.9820019	total: 16.7ms	remaining: 3.03s
11:	learn: 26750.1436307	total: 18.1ms	remaining: 2.99s
12:	learn: 25933.8771875	total: 19.2ms	remaining: 2.94s
13:	learn: 25276.2209603	total: 20.4ms	remaining: 2.89s
14:	learn: 24814.8840007	total: 21.5ms	remaining: 2.85s
15:	learn: 24148.9022537	total: 22.7ms	remaining: 2.81s
16:	learn: 23736.2318493	total: 23.9ms	remaining: 2.79s
17:	learn: 23130.3251115	total: 25.1ms	remaining: 2.76s
18

198:	learn: 7091.0904196	total: 290ms	remaining: 2.62s
199:	learn: 7086.0565805	total: 292ms	remaining: 2.63s
200:	learn: 7073.1932687	total: 294ms	remaining: 2.63s
201:	learn: 7054.9608525	total: 295ms	remaining: 2.63s
202:	learn: 7017.0330505	total: 297ms	remaining: 2.63s
203:	learn: 6994.9222403	total: 298ms	remaining: 2.62s
204:	learn: 6953.2451623	total: 299ms	remaining: 2.62s
205:	learn: 6922.0839511	total: 301ms	remaining: 2.62s
206:	learn: 6897.7234802	total: 303ms	remaining: 2.62s
207:	learn: 6865.5074470	total: 304ms	remaining: 2.62s
208:	learn: 6813.9304544	total: 306ms	remaining: 2.62s
209:	learn: 6781.1096119	total: 308ms	remaining: 2.62s
210:	learn: 6758.1546911	total: 309ms	remaining: 2.62s
211:	learn: 6724.6810587	total: 311ms	remaining: 2.62s
212:	learn: 6692.0776142	total: 312ms	remaining: 2.62s
213:	learn: 6657.1987324	total: 313ms	remaining: 2.62s
214:	learn: 6631.5302823	total: 315ms	remaining: 2.62s
215:	learn: 6593.1116206	total: 318ms	remaining: 2.63s
216:	learn

413:	learn: 3405.1132679	total: 589ms	remaining: 2.25s
414:	learn: 3389.5611774	total: 591ms	remaining: 2.25s
415:	learn: 3381.0253019	total: 592ms	remaining: 2.25s
416:	learn: 3360.5876870	total: 593ms	remaining: 2.25s
417:	learn: 3349.8918716	total: 594ms	remaining: 2.25s
418:	learn: 3341.7557882	total: 595ms	remaining: 2.25s
419:	learn: 3328.4231339	total: 597ms	remaining: 2.25s
420:	learn: 3323.2201587	total: 598ms	remaining: 2.24s
421:	learn: 3321.1026171	total: 600ms	remaining: 2.24s
422:	learn: 3309.7512743	total: 601ms	remaining: 2.24s
423:	learn: 3298.3482648	total: 603ms	remaining: 2.24s
424:	learn: 3290.3832179	total: 604ms	remaining: 2.24s
425:	learn: 3285.2454601	total: 606ms	remaining: 2.24s
426:	learn: 3275.7835110	total: 607ms	remaining: 2.23s
427:	learn: 3265.3775117	total: 608ms	remaining: 2.23s
428:	learn: 3246.3999907	total: 609ms	remaining: 2.23s
429:	learn: 3240.7200803	total: 611ms	remaining: 2.23s
430:	learn: 3229.7815839	total: 612ms	remaining: 2.23s
431:	learn

642:	learn: 1776.2174201	total: 885ms	remaining: 1.87s
643:	learn: 1770.3204039	total: 887ms	remaining: 1.87s
644:	learn: 1762.9085745	total: 889ms	remaining: 1.87s
645:	learn: 1755.7470320	total: 890ms	remaining: 1.86s
646:	learn: 1754.1241934	total: 892ms	remaining: 1.86s
647:	learn: 1750.1943525	total: 893ms	remaining: 1.86s
648:	learn: 1745.9969202	total: 894ms	remaining: 1.86s
649:	learn: 1744.1107038	total: 896ms	remaining: 1.86s
650:	learn: 1737.6557626	total: 897ms	remaining: 1.86s
651:	learn: 1729.9934954	total: 899ms	remaining: 1.86s
652:	learn: 1728.8413510	total: 900ms	remaining: 1.86s
653:	learn: 1722.3033547	total: 902ms	remaining: 1.86s
654:	learn: 1708.9792274	total: 903ms	remaining: 1.85s
655:	learn: 1701.6765220	total: 905ms	remaining: 1.85s
656:	learn: 1699.3863134	total: 906ms	remaining: 1.85s
657:	learn: 1695.0105325	total: 908ms	remaining: 1.85s
658:	learn: 1688.9487888	total: 910ms	remaining: 1.85s
659:	learn: 1687.7019446	total: 911ms	remaining: 1.85s
660:	learn

853:	learn: 1031.5670718	total: 1.19s	remaining: 1.59s
854:	learn: 1029.5240262	total: 1.19s	remaining: 1.59s
855:	learn: 1028.0184079	total: 1.19s	remaining: 1.59s
856:	learn: 1025.9197879	total: 1.19s	remaining: 1.59s
857:	learn: 1025.4788452	total: 1.19s	remaining: 1.59s
858:	learn: 1023.7189335	total: 1.2s	remaining: 1.59s
859:	learn: 1020.8354191	total: 1.2s	remaining: 1.59s
860:	learn: 1015.7247626	total: 1.2s	remaining: 1.59s
861:	learn: 1010.6033074	total: 1.2s	remaining: 1.59s
862:	learn: 1010.1487007	total: 1.2s	remaining: 1.59s
863:	learn: 1006.9729700	total: 1.21s	remaining: 1.58s
864:	learn: 1006.1204799	total: 1.21s	remaining: 1.59s
865:	learn: 1004.4527706	total: 1.21s	remaining: 1.58s
866:	learn: 1003.1913150	total: 1.21s	remaining: 1.58s
867:	learn: 997.9676692	total: 1.21s	remaining: 1.58s
868:	learn: 994.6706805	total: 1.22s	remaining: 1.58s
869:	learn: 992.6116798	total: 1.22s	remaining: 1.58s
870:	learn: 986.6910347	total: 1.22s	remaining: 1.58s
871:	learn: 985.632

1073:	learn: 582.4545084	total: 1.48s	remaining: 1.28s
1074:	learn: 582.0097486	total: 1.48s	remaining: 1.28s
1075:	learn: 581.5415874	total: 1.49s	remaining: 1.28s
1076:	learn: 580.7482184	total: 1.49s	remaining: 1.28s
1077:	learn: 577.6752279	total: 1.49s	remaining: 1.27s
1078:	learn: 573.8522524	total: 1.49s	remaining: 1.27s
1079:	learn: 571.0498704	total: 1.49s	remaining: 1.27s
1080:	learn: 569.5293537	total: 1.49s	remaining: 1.27s
1081:	learn: 562.8597251	total: 1.5s	remaining: 1.27s
1082:	learn: 562.1972529	total: 1.5s	remaining: 1.27s
1083:	learn: 561.6305245	total: 1.5s	remaining: 1.27s
1084:	learn: 559.9895070	total: 1.5s	remaining: 1.26s
1085:	learn: 559.1259689	total: 1.5s	remaining: 1.26s
1086:	learn: 557.9325157	total: 1.5s	remaining: 1.26s
1087:	learn: 556.1907205	total: 1.5s	remaining: 1.26s
1088:	learn: 554.6392218	total: 1.51s	remaining: 1.26s
1089:	learn: 554.0914731	total: 1.51s	remaining: 1.26s
1090:	learn: 552.7978358	total: 1.51s	remaining: 1.26s
1091:	learn: 551.

1309:	learn: 335.9447953	total: 1.78s	remaining: 939ms
1310:	learn: 335.6320389	total: 1.78s	remaining: 938ms
1311:	learn: 335.3217889	total: 1.78s	remaining: 937ms
1312:	learn: 334.1670815	total: 1.79s	remaining: 935ms
1313:	learn: 332.6879138	total: 1.79s	remaining: 934ms
1314:	learn: 331.4994380	total: 1.79s	remaining: 933ms
1315:	learn: 331.2726664	total: 1.79s	remaining: 931ms
1316:	learn: 330.3075176	total: 1.79s	remaining: 930ms
1317:	learn: 329.9487692	total: 1.79s	remaining: 929ms
1318:	learn: 329.5573946	total: 1.8s	remaining: 928ms
1319:	learn: 329.2486275	total: 1.8s	remaining: 926ms
1320:	learn: 328.9988643	total: 1.8s	remaining: 925ms
1321:	learn: 328.3828028	total: 1.8s	remaining: 923ms
1322:	learn: 326.8471704	total: 1.8s	remaining: 922ms
1323:	learn: 326.2523230	total: 1.8s	remaining: 921ms
1324:	learn: 326.0721918	total: 1.8s	remaining: 920ms
1325:	learn: 324.7440343	total: 1.81s	remaining: 919ms
1326:	learn: 323.7468529	total: 1.81s	remaining: 917ms
1327:	learn: 322.

1546:	learn: 189.0756792	total: 2.08s	remaining: 609ms
1547:	learn: 188.6241629	total: 2.08s	remaining: 608ms
1548:	learn: 187.6041215	total: 2.08s	remaining: 607ms
1549:	learn: 186.8715199	total: 2.09s	remaining: 606ms
1550:	learn: 186.3743928	total: 2.09s	remaining: 604ms
1551:	learn: 185.8290238	total: 2.09s	remaining: 603ms
1552:	learn: 185.1029797	total: 2.09s	remaining: 602ms
1553:	learn: 184.9920554	total: 2.09s	remaining: 600ms
1554:	learn: 184.8257900	total: 2.09s	remaining: 599ms
1555:	learn: 184.2080174	total: 2.09s	remaining: 598ms
1556:	learn: 183.9486551	total: 2.1s	remaining: 596ms
1557:	learn: 183.3692979	total: 2.1s	remaining: 595ms
1558:	learn: 182.8805040	total: 2.1s	remaining: 594ms
1559:	learn: 182.7899109	total: 2.1s	remaining: 593ms
1560:	learn: 182.2035517	total: 2.1s	remaining: 591ms
1561:	learn: 182.0022158	total: 2.1s	remaining: 590ms
1562:	learn: 181.3044173	total: 2.11s	remaining: 589ms
1563:	learn: 180.8939915	total: 2.11s	remaining: 587ms
1564:	learn: 180

1777:	learn: 114.0642952	total: 2.38s	remaining: 298ms
1778:	learn: 113.6840286	total: 2.38s	remaining: 296ms
1779:	learn: 113.5025347	total: 2.38s	remaining: 295ms
1780:	learn: 113.0166725	total: 2.39s	remaining: 294ms
1781:	learn: 112.7549057	total: 2.39s	remaining: 292ms
1782:	learn: 112.2131222	total: 2.39s	remaining: 291ms
1783:	learn: 111.7300473	total: 2.39s	remaining: 289ms
1784:	learn: 111.3939543	total: 2.39s	remaining: 288ms
1785:	learn: 111.1860878	total: 2.39s	remaining: 287ms
1786:	learn: 111.1421570	total: 2.39s	remaining: 285ms
1787:	learn: 110.7767300	total: 2.4s	remaining: 284ms
1788:	learn: 110.4598467	total: 2.4s	remaining: 283ms
1789:	learn: 110.1269409	total: 2.4s	remaining: 281ms
1790:	learn: 110.0341504	total: 2.4s	remaining: 280ms
1791:	learn: 109.7975762	total: 2.4s	remaining: 279ms
1792:	learn: 109.5000196	total: 2.4s	remaining: 277ms
1793:	learn: 109.1847545	total: 2.4s	remaining: 276ms
1794:	learn: 108.9040471	total: 2.4s	remaining: 275ms
1795:	learn: 108.5

In [97]:
X_te_test = X_te_test.fillna(0)

In [98]:
X_te_test.iloc[:,43]

832     189319
965     189319
969     189319
1411    189319
284     189319
         ...  
703     189319
661     189319
798     189319
351     189319
521     189319
Name: Utilities_Kfold_Target_Enc, Length: 442, dtype: object

In [99]:
X_te_test == 'NoSeWa'

C:\Users\Anastasia\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Electrical_Kfold_Target_Enc,KitchenQual_Kfold_Target_Enc,Functional_Kfold_Target_Enc,GarageType_Kfold_Target_Enc,GarageFinish_Kfold_Target_Enc,GarageQual_Kfold_Target_Enc,GarageCond_Kfold_Target_Enc,PavedDrive_Kfold_Target_Enc,SaleType_Kfold_Target_Enc,SaleCondition_Kfold_Target_Enc
832,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
965,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
969,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1411,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
284,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
661,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
798,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
351,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [100]:
X_te_test = X_te_test.replace('NoSeWa',0.2989737894613584) 
X_te_test = X_te_test.replace('Roll',0.2989737894613584) 
X_te_test = X_te_test.replace('ClyTile',0.2989737894613584) 

In [101]:
X_te_test.iloc[47,43]

0.2989737894613584

In [102]:
X_te_test.to_csv("X_te_test")


PermissionError: [Errno 13] Permission denied: 'X_te_test'

In [103]:
X_te_test.loc[: , "gt"]=y_test

In [109]:
predictions = []
y_test_new= []
for idx,row in X_te_test.iterrows():
    try:
        pred =model.predict([row])
    except:
        continue
    else:
        predictions.append(pred[0])
        y_test_new.append(row["gt"]) 

In [110]:
MAE=mae(predictions,y_test_new
       )

In [111]:
 MAE 

21025.36027686617